In [181]:
import gym

In [182]:
env = gym.make("CartPole-v0")

In [183]:
env

<TimeLimit<CartPoleEnv<CartPole-v0>>>

In [188]:
env.observation_space.shape[0]

4

In [189]:
env.action_space.n

2

In [4]:
env.reset()

array([-0.04950953,  0.01081784, -0.01487868,  0.00689617])

In [5]:
env.action_space

Discrete(2)

In [6]:
env.observation_space

Box(4,)

In [7]:
env.step(0)

(array([-0.04929318, -0.18408761, -0.01474076,  0.29484781]), 1.0, False, {})

In [8]:
env.action_space.sample()

0

In [81]:
total_reward = 0.0
total_steps = 0

obs = env.reset()
while True:
    action = env.action_space.sample()
    obs, reward, done, _ = env.step(action)
    total_reward += reward
    total_steps += 1
    if done:
        break
print("Episode done in {:d} steps, total reward {:.2f}".format(total_steps,total_reward))

Episode done in 14 steps, total reward 14.00


In [29]:
class Custom_Wrapper(object):
    def __init__(self):
        print("Init Custom_Wrapper")
        
    atr_1 = 15
        
class Sub_Custom_Wrapper(Custom_Wrapper):
    def __init__(self):
        
        print("Init Sub_Custom_Wrapper")
        super().__init__()
    
    def run_me(self):
        print('you run me...')
    
    @property
    def prop_go(self):
        return 10
    
sub1 = Sub_Custom_Wrapper()
sub2 = Sub_Custom_Wrapper()
Sub_Custom_Wrapper.atr_1 = 17


Init Sub_Custom_Wrapper
Init Custom_Wrapper
Init Sub_Custom_Wrapper
Init Custom_Wrapper


In [30]:
print(sub1.atr_1)
print(sub2.atr_1)

17
17


In [31]:
sub1.run_me()

you run me...


In [32]:
sub1.prop_go

10

In [15]:
gym.Env??

In [2]:
from torch.utils.tensorboard import SummaryWriter
import math

In [14]:
writer = SummaryWriter()

In [15]:
funcs = {"sin" : math.sin, "cos" : math.cos, "tan" : math.tan}

In [16]:
for angle in range(-360,360):
    angle_rad = angle * math.pi / 180.0
    for name, fun in funcs.items():
        val = fun(angle_rad)
        writer.add_scalar(name,val,angle)
writer.close()

In [26]:
import math

In [32]:
funcs = (lambda x: 2*math.sin(x), math.cos)

In [34]:
[f(math.pi/2.0) for f in funcs]

[2.0, 6.123233995736766e-17]

In [35]:
import numpy as np

In [180]:
act_probs = np.array([0.1,0.8,0.1])
np.random.choice(len(act_probs), p=act_probs)

1

In [91]:
act_probs

array([0.3, 0.5, 0.2])

In [192]:
import torch
torch.FloatTensor([(0.2,0.4,0.5,3.2),(2.,5,5,6)])

tensor([[0.2000, 0.4000, 0.5000, 3.2000],
        [2.0000, 5.0000, 5.0000, 6.0000]])

In [194]:
#!/usr/bin/env python3
import gym
from collections import namedtuple
import numpy as np
from torch.utils.tensorboard import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim


HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70


class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)


Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])


def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=obs, action=action))
        if is_done:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs


def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_obs = []
    train_act = []
    for example in batch:
        if example.reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, example.steps))
        train_act.extend(map(lambda step: step.action, example.steps))

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean


if __name__ == "__main__":
    env = gym.make("CartPole-v0")
    # env = gym.wrappers.Monitor(env, directory="mon", force=True)
    obs_size = env.observation_space.shape[0]
    n_actions = env.action_space.n

    net = Net(obs_size, HIDDEN_SIZE, n_actions)
    objective = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params=net.parameters(), lr=0.01)
    writer = SummaryWriter(comment="-cartpole")

    for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
        obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
        optimizer.zero_grad()
        action_scores_v = net(obs_v)
        loss_v = objective(action_scores_v, acts_v)
        loss_v.backward()
        optimizer.step()
        print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (
            iter_no, loss_v.item(), reward_m, reward_b))
        writer.add_scalar("loss", loss_v.item(), iter_no)
        writer.add_scalar("reward_bound", reward_b, iter_no)
        writer.add_scalar("reward_mean", reward_m, iter_no)
        if reward_m > 199:
            print("Solved!")
            break
    writer.close()

0: loss=0.686, reward_mean=18.2, reward_bound=19.5
1: loss=0.681, reward_mean=26.5, reward_bound=29.5
2: loss=0.664, reward_mean=31.5, reward_bound=38.0
3: loss=0.660, reward_mean=36.1, reward_bound=45.0
4: loss=0.635, reward_mean=37.2, reward_bound=49.0
5: loss=0.625, reward_mean=46.6, reward_bound=49.0
6: loss=0.630, reward_mean=47.8, reward_bound=49.0
7: loss=0.630, reward_mean=51.8, reward_bound=56.0
8: loss=0.611, reward_mean=62.1, reward_bound=65.0
9: loss=0.614, reward_mean=64.9, reward_bound=85.0
10: loss=0.605, reward_mean=55.8, reward_bound=67.5
11: loss=0.592, reward_mean=50.3, reward_bound=54.5
12: loss=0.590, reward_mean=56.6, reward_bound=64.5
13: loss=0.569, reward_mean=77.6, reward_bound=90.0
14: loss=0.563, reward_mean=82.1, reward_bound=97.5
15: loss=0.578, reward_mean=86.1, reward_bound=100.0
16: loss=0.578, reward_mean=92.4, reward_bound=112.5
17: loss=0.568, reward_mean=92.0, reward_bound=104.5
18: loss=0.582, reward_mean=79.2, reward_bound=83.5
19: loss=0.551, rew

In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.get_device_name()

'GeForce GT 740M'

In [4]:
torch.backends.cudnn.enabled

True

In [7]:
a = torch.randn(3,3).to('cuda')

In [8]:
a

tensor([[-0.4151, -0.4766,  1.1099],
        [ 0.5927, -0.8553, -0.2762],
        [ 0.9678,  1.6284,  0.6820]], device='cuda:0')

In [15]:
a_n = a.to('cpu').data.numpy()

In [16]:
a_n

array([[-0.4150756 , -0.4765675 ,  1.1099129 ],
       [ 0.5926538 , -0.85533494, -0.27618036],
       [ 0.9678113 ,  1.6283945 ,  0.6819889 ]], dtype=float32)

In [17]:
device = torch.device('cpu')

In [18]:
device

device(type='cpu')